In [12]:
from typing import List
from itertools import accumulate
from time import perf_counter_ns
import numpy as np
from scipy.sparse import csc_matrix, kron, identity
import cirq
import openfermion as of
from openfermionpyscf import run_pyscf
import quimb.tensor as qtn
from quimb.tensor.tensor_1d import MatrixProductState, MatrixProductOperator
from convert import to_groups_of
from error_pert import get_v2_sarray
from qpe_trotter import (
    v2_pauli_sum,
    v2_qubop,
    to_groups_mpo,
    get_v2_contrib_mpo,
    build_v2_terms,
    compute_expectation_parallel,
    compute_expectation_sequential
)
from kcommute import get_si_sets
from tensor_network_common import pauli_sum_to_mpo, mps_to_vector
from qtoolbox.converters.openfermion_bridge import from_openfermion
from qtoolbox.core.hamiltonian import Hamiltonian
from qtoolbox.grouping import sorted_insertion_grouping

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
hamiltonian_file = "data/monomer_eqb.hdf5"
hamiltonian = of.jordan_wigner(
    of.get_fermion_operator(
        of.chem.MolecularData(filename=hamiltonian_file).get_molecular_hamiltonian()
    )
)

nq = of.utils.count_qubits(hamiltonian)
nterms = len(hamiltonian.terms)
print(f"Hamiltonian acts on {nq} qubits and has {nterms} terms.")
hamiltonian_psum = of.transforms.qubit_operator_to_pauli_sum(hamiltonian)

Hamiltonian acts on 14 qubits and has 1620 terms.


In [5]:
qs = cirq.LineQubit.range(nq)
hamiltonian_mpo = pauli_sum_to_mpo(hamiltonian_psum, qs, 100)
dmrg = qtn.DMRG(hamiltonian_mpo, bond_dims=15)
converged = dmrg.solve()
if not converged:
    print("DMRG did not converge.")
ground_state = dmrg.state
ground_state_vec = mps_to_vector(ground_state)

/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/cotengra/hyperoptimizers/hyper.py:55: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization. It is recommended to install one of these libraries for higher quality contraction paths.
  warnings.warn(
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: divide by zero encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: overflow encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: invalid value encountered in matmul
  return func(*args, **kwargs)


sorted_inds = ['k0', 'k1', 'k2', 'k3', 'k4', 'k5', 'k6', 'k7', 'k8', 'k9', 'k10', 'k11', 'k12', 'k13']


/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: divide by zero encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: overflow encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: invalid value encountered in matmul
  return func(*args, **kwargs)


In [6]:
start_time = perf_counter_ns()
groups = get_si_sets(hamiltonian_psum, nq)
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"There are {len(groups)} groups.")
print(f"Elapsed time: {elapsed_time:4.5e} ns")

There are 65 groups.
Elapsed time: 7.16897e+09 ns


## Sparse matrices

In [7]:
# Use the code from the paper.
start_time = perf_counter_ns()
group_qubops = to_groups_of(groups)
sparse_frag_ops = []
# Convert group operators to sparse matrices.
# We must make sure the matrices have all the same size.
# If a matrix is not big enough, tensor it with I on the right.
for op in group_qubops:
    nq_op = of.utils.count_qubits(op)
    op_sparse = of.linalg.get_sparse_operator(op)
    if nq_op != nq:
        eye_diff = identity(2 ** (nq - nq_op), dtype="complex", format='csc')
        new_op = kron(op_sparse, eye_diff, format="csc")
        sparse_frag_ops.append(new_op)
    else:
        sparse_frag_ops.append(op_sparse)
v2_sparse = get_v2_sarray(sparse_frag_ops)
eps2 = np.vdot(ground_state_vec, v2_sparse @ ground_state_vec).real
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"Got eps2={eps2} in {elapsed_time:4.5e} ns")

Got eps2=0.07948781131434035 in 5.45826e+10 ns


## Quantum toolbox

In [13]:
# Convert OpenFermion QubitOperator to quantum-toolbox Hamiltonian
n_qubits_qt = of.utils.count_qubits(hamiltonian)
terms = [from_openfermion(term, coeff, n_qubits_qt)
         for term, coeff in hamiltonian.terms.items() if term]  # skip identity
ham = Hamiltonian(terms)
print(f"Loaded Hamiltonian: {ham.num_terms()} terms, {ham.num_qubits()} qubits")

# Group using quantum-toolbox's SI
start_group = perf_counter_ns()
group_collection = sorted_insertion_grouping(ham)
time_grouping = perf_counter_ns() - start_group

sym_groups = [list(g.paulis) for g in group_collection.groups]
print(f"Grouped into {len(sym_groups)} groups in {time_grouping/1e9:.3f} s (native symplectic format)")

start_time = perf_counter_ns()
v2_terms = build_v2_terms(sym_groups)
time_build = perf_counter_ns() - start_time

start_exp = perf_counter_ns()
eps2_symplectic = compute_expectation_sequential(v2_terms, ground_state_vec, nq)
time_exp = perf_counter_ns() - start_exp

end_time = perf_counter_ns()
elapsed_ns = end_time - start_time

error = abs(eps2 - eps2_symplectic)
print(f"quantum-toolbox (sequential):")
print(f"   V2 terms:     {len(v2_terms):,}")
print(f"   eps2 =        {eps2_symplectic:.10f}")
print(f"   Build time:   {time_build/1e9:.3f} s")
print(f"   Exp. time:    {time_exp/1e9:.3f} s")
print(f"   Total time:   {elapsed_ns/1e9:.3f} s")
print(f"   Error vs sparse: {error:.2e}")
assert error < 1e-8, f"Results don't match! Error: {error}"
print("   ✓ Validation passed")

start_time = perf_counter_ns()
v2_terms = build_v2_terms(sym_groups)
time_build = perf_counter_ns() - start_time
print(f"   Build time:   {time_build/1e9:.3f} s")
n_workers = 9
print(f"quantum-toolbox (parallel, {n_workers} workers):")

start_time = perf_counter_ns()
eps2_parallel = compute_expectation_parallel(v2_terms, ground_state_vec, nq, n_workers)
time_parallel = perf_counter_ns() - start_time

error_par = abs(eps2 - eps2_parallel)
print(f"   eps2 =        {eps2_parallel:.10f}")
print(f"   Time:         {time_parallel/1e9:.3f} s")
assert error_par < 1e-8, f"Results don't match! Error: {error_par}"
print("   ✓ Validation passed")

Loaded Hamiltonian: 1619 terms, 14 qubits
Grouped into 65 groups in 0.027 s (native symplectic format)


KeyboardInterrupt: 

## Tensor networks

In [9]:
max_mpo_bond = 5
start_time = perf_counter_ns()
mpo_fragments = to_groups_mpo(groups, qs, max_mpo_bond)
end_time = perf_counter_ns()
elapsed_time_convert = end_time - start_time

start_time = perf_counter_ns()
eps2_mpo = get_v2_contrib_mpo(mpo_fragments, ground_state, max_mpo_bond)
end_time = perf_counter_ns()
elapsed_time_calculate = end_time - start_time

print(f"Got eps2={eps2_mpo}.")
print(f"Conversion time: {elapsed_time_convert:4.5e}, calculation time: {elapsed_time_calculate:4.5e}.")
eps2_err = abs(eps2 - eps2_mpo)
print(f"Error {eps2_err:4.5e}")

/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: divide by zero encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: overflow encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: invalid value encountered in matmul
  return func(*args, **kwargs)


Got eps2=0.2730535231619882.
Conversion time: 3.18974e+09, calculation time: 9.21265e+11.
Error 1.93566e-01
